### Data collection

This notebook is used to collect data on polynomials that have low Mahler measure.

In [39]:
# imports
import pandas as pd
import csv

This is more of a temporary solution, but I'm going to paste the code from the sage file here since I can't import it as a Python module:

In [57]:
def boundedReciprocalPolys(d, M):
    possiblePoly = []
    assert d % 2 == 0
    n = d / 2
    # create a ring to make polynomials (for testing whether the polynomial is reducible)
    R.<x> = PolynomialRing(QQ)
    # calculate the bounds for each coefficient
    bounds = [M * binomial(d, i) for i in range(0, n+1)]
    # this line is for testing v
    # print("bounds: ")
    # print(bounds)

    # this is done to account for the fact that the xmrange function always starts at 0
    tempBounds = [2 * bound + 1 for bound in bounds]
    possibleCoef = list(xmrange(tempBounds))
    
    print("test")

    # shift the values back to fix the coefficients, and add the remaining coefficients
    # (for the terms of degree n + 1 or higher)
    # these are known since the polynomial is reciprocal
    for coefs in possibleCoef:
        fixedCoefs = [int(coefs[i] - bounds[i]) for i in range(len(bounds))]
        remainingCoefs = [fixedCoefs[n - (i + 1)] for i in range(n)]
        fixedCoefs.extend(remainingCoefs)
        possiblePoly.append(fixedCoefs)

    # filter out all the polynomials with 0 as the first coefficient (since this would mean the
    # degree of the polynomial would be less than d)
    for polyCoefs in possiblePoly:
        if polyCoefs[0] == 0:
            possiblePoly.remove(polyCoefs) # this might be slow, might there be a better way to do this?

    # and filter out any reducible polynomials as well

    # convert the 
    possiblePoly = convertToPolys(possiblePoly)

    filteredPolys = filterOutReducibles(possiblePoly)

    return filteredPolys

def lowMahlerPolys(boundedPolys, M):
    lowPolys = []
    for poly in boundedPolys:
        measure = mahlerMeasure(poly)
        # filter out polys with higher Mahler measure than M or equal to 1
        if measure <= M and measure != 1:
            lowPolys.append(poly)
    
    return lowPolys

# converts a list of polynomial coefficients to a polynomial object
def convertToPolys(polyCoefs):
    convertedList = []
    for coefs in polyCoefs:
        convertedList.append(convertToPoly(coefs))
    return convertedList

# filters out the reducibles from the list of polynomial objects
def filterOutReducibles(polys):
    newPolys = []
    for poly in polys:
        if poly.is_irreducible():
            newPolys.append(poly)
    return newPolys


# computes the galois group for the given polynomial.
def getGalois(poly):
    # make sure the polynomial is not reducible before
    # computing its Galois group
    # assert poly.is_irreducible
    K.<a> = NumberField(poly)
    G = K.galois_group()
    return G

def convertToPoly(coefs):
    # Define a polynomial ring for our polynomials and convert coefs to poly using this ring
    R = PolynomialRing(QQ, 'x')
    return R(coefs)


# function to get the roots and Mahler measure

#Returns roots w norm greater than 1
def bigRoots(poly): 
    S.<x> = PolynomialRing(CC)
    tempPoly = S(poly)
    rootList = tempPoly.roots()
    bigRootList = []
    for r in rootList:
        if sqrt(norm(r[0])) > 1:
            bigRootList.append(r)

    return bigRootList

def realRoots(poly): 
    rootList = list(poly.roots())
    realRootList = []
    for r in rootList:
        if r[0] == conjugate(r[0]):
            realRootList.append(r)

    return realRootList

def mahlerMeasure(poly):
    S.<x> = PolynomialRing(CC)
    tempPoly = S(poly)
    rootList = list(tempPoly.roots())
    measure = 1
    
    # print(rootList)
    
    # question: are we dealing with multiplicity correctly?
    for s in rootList:
        s_2 = sqrt(norm(s[0]))
        measure = measure * max(1, s_2)

    return measure

#Returns trace polynomial 
def tracePoly(poly):
    K.<a> = NumberField(poly)
    return (a + 1/a).minpoly()

# one goal: plot the number of polynomials with Mahler Measure under a certain bound
# as you increase the degree

# things we want to collect:
# ((f) poly, roots of f outside unit circle, # of real roots, (g) trace poly, G_f, G_g, Mahler, disc)
# collect data into a csv (so we can plot it)

def getStats(poly):
    # poly, degree, Mahler measure, roots outside unit circle, # real roots, trace poly, galois of poly, galois of trace, discriminant, 
    trace = tracePoly(poly)
    return [poly 
            , poly.degree()
            , mahlerMeasure(poly) 
            , bigRoots(poly) # roots outside unit circle
            , realRoots(poly) 
            , trace
            , getGalois(poly)
            , getGalois(trace)
            , disc(poly) # discriminant
          ] 

# decomposition groups

# polys = boundedReciprocalPolys(4, 1.3)

# print(polys)
# 
# k = 1
# for polyCoefs in polys:
#     print("Polynomial " + str(k) + ":")
#     print(polyCoefs)
#     poly = convertToPoly(polyCoefs)
#     print(poly)
#     #try:
#     print("irreducible: " + str(poly.is_irreducible()))
#     if not poly.is_irreducible():
#         continue
#     print(getStats(convertToPoly(polyCoefs)))
#     
#     #except:
#     #        print("Polynomial is reducible.")
#     k += 1


Now I'm going to gather all the contenders for polynomials with Mahler measure below a certain bound and of degree d:

In [58]:
d = 4
M = 3.5
polys = boundedReciprocalPolys(d, M)
# for poly in polys:
#     print(mahlerMeasure(poly))
lowMahler = lowMahlerPolys(polys, M)
print(lowMahler)

test
[-3*x^4 - 14*x^3 - 20*x^2 - 14*x - 3, -3*x^4 - 14*x^3 - 19*x^2 - 14*x - 3, -3*x^4 - 14*x^3 - 18*x^2 - 14*x - 3, -3*x^4 - 13*x^3 - 21*x^2 - 13*x - 3, -3*x^4 - 13*x^3 - 19*x^2 - 13*x - 3, -3*x^4 - 13*x^3 - 17*x^2 - 13*x - 3, -3*x^4 - 13*x^3 - 15*x^2 - 13*x - 3, -3*x^4 - 13*x^3 - 14*x^2 - 13*x - 3, -3*x^4 - 13*x^3 - 13*x^2 - 13*x - 3, -3*x^4 - 12*x^3 - 17*x^2 - 12*x - 3, -3*x^4 - 12*x^3 - 16*x^2 - 12*x - 3, -3*x^4 - 12*x^3 - 14*x^2 - 12*x - 3, -3*x^4 - 12*x^3 - 13*x^2 - 12*x - 3, -3*x^4 - 12*x^3 - 12*x^2 - 12*x - 3, -3*x^4 - 12*x^3 - 11*x^2 - 12*x - 3, -3*x^4 - 12*x^3 - 10*x^2 - 12*x - 3, -3*x^4 - 12*x^3 - 9*x^2 - 12*x - 3, -3*x^4 - 11*x^3 - 18*x^2 - 11*x - 3, -3*x^4 - 11*x^3 - 17*x^2 - 11*x - 3, -3*x^4 - 11*x^3 - 15*x^2 - 11*x - 3, -3*x^4 - 11*x^3 - 13*x^2 - 11*x - 3, -3*x^4 - 11*x^3 - 11*x^2 - 11*x - 3, -3*x^4 - 11*x^3 - 10*x^2 - 11*x - 3, -3*x^4 - 11*x^3 - 9*x^2 - 11*x - 3, -3*x^4 - 11*x^3 - 8*x^2 - 11*x - 3, -3*x^4 - 11*x^3 - 7*x^2 - 11*x - 3, -3*x^4 - 11*x^3 - 5*x^2 - 11*x - 3, 

Now we can compute several things for these polynomials and gather them together:

In [61]:
headers = ["polynomial", "degree", "Mahler measure", "roots outside unit circle", 
           "number of real roots", "trace poly", "Galois of poly", "Galois of trace", "discriminant"]
polyData = [getStats(poly) for poly in polys]
print(len(polyData))
df = pd.DataFrame(polyData)
df.columns = headers
df

6472


,polynomial,degree,Mahler measure,roots outside unit circle,number of real roots,trace poly,Galois of poly,Galois of trace,discriminant
0,-3*x^4 - 14*x^3 - 20*x^2 - 14*x - 3,4,2.86637811053993,"[(-2.86637811053993, 1)]",[],x^2 + 14/3*x + 14/3,"((), (1,2)(3,7)(4,6)(5,8), (1,3)(2,5)(4,7)(6,8...","((), (1,2))",-84672
1,-3*x^4 - 14*x^3 - 19*x^2 - 14*x - 3,4,3.06070362895875,"[(-3.06070362895875, 1)]",[],x^2 + 14/3*x + 13/3,"((), (1,2)(3,4)(5,6)(7,8), (1,3)(2,4)(5,7)(6,8...","((), (1,2))",-254400
2,-3*x^4 - 14*x^3 - 18*x^2 - 14*x - 3,4,3.22511753993383,"[(-3.22511753993383, 1)]",[],x^2 + 14/3*x + 4,"((), (1,2)(3,5)(4,8)(6,7), (1,3)(2,5)(4,7)(6,8...","((), (1,2))",-562432
3,-3*x^4 - 14*x^3 - 16*x^2 - 14*x - 3,4,3.50063738400638,"[(-3.50063738400638, 1)]",[],x^2 + 14/3*x + 10/3,"((), (1,2)(3,4)(5,8)(6,7), (1,3)(2,4)(5,6)(7,8...","((), (1,2))",-1732800
4,-3*x^4 - 14*x^3 - 15*x^2 - 14*x - 3,4,3.62060859346276,"[(-3.62060859346276, 1)]",[],x^2 + 14/3*x + 3,"((), (1,2,3,5)(4,8,6,7), (1,3)(2,5)(4,6)(7,8),...","((), (1,2))",-2656192
...,...,...,...,...,...,...,...,...,...
6467,3*x^4 + 14*x^3 + 15*x^2 + 14*x + 3,4,3.62060859346276,"[(-3.62060859346276, 1)]",[],x^2 + 14/3*x + 3,"((), (1,2,3,5)(4,8,6,7), (1,3)(2,5)(4,6)(7,8),...","((), (1,2))",-2656192
6468,3*x^4 + 14*x^3 + 16*x^2 + 14*x + 3,4,3.50063738400638,"[(-3.50063738400638, 1)]",[],x^2 + 14/3*x + 10/3,"((), (1,2)(3,4)(5,8)(6,7), (1,3)(2,4)(5,6)(7,8...","((), (1,2))",-1732800
6469,3*x^4 + 14*x^3 + 18*x^2 + 14*x + 3,4,3.22511753993383,"[(-3.22511753993383, 1)]",[],x^2 + 14/3*x + 4,"((), (1,2)(3,5)(4,8)(6,7), (1,3)(2,5)(4,7)(6,8...","((), (1,2))",-562432
6470,3*x^4 + 14*x^3 + 19*x^2 + 14*x + 3,4,3.06070362895875,"[(-3.06070362895875, 1)]",[],x^2 + 14/3*x + 13/3,"((), (1,2)(3,4)(5,6)(7,8), (1,3)(2,4)(5,7)(6,8...","((), (1,2))",-254400


In [66]:
df[df["Mahler measure"] != 1].sort_values("Mahler measure")

,polynomial,degree,Mahler measure,roots outside unit circle,number of real roots,trace poly,Galois of poly,Galois of trace,discriminant
419,-3*x^4 - 3*x^3 - 7*x^2 - 3*x - 3,4,1.34559448553445,"[(-0.286834423817264 - 1.12397531060423*I, 1),...",[],x^2 + x + 1/3,"((), (1,2,6,4)(3,7,5,8), (1,3)(2,8)(4,7)(5,6),...","((), (1,2))",1197
6052,3*x^4 + 3*x^3 + 7*x^2 + 3*x + 3,4,1.34559448553445,"[(-0.286834423817264 - 1.12397531060423*I, 1),...",[],x^2 + x + 1/3,"((), (1,2,6,4)(3,7,5,8), (1,3)(2,8)(4,7)(5,6),...","((), (1,2))",1197
5831,3*x^4 - 3*x^3 + 7*x^2 - 3*x + 3,4,1.34559448553445,"[(0.286834423817264 - 1.12397531060423*I, 1), ...",[],x^2 - x + 1/3,"((), (1,2,7,8)(3,6,5,4), (1,3)(2,4)(5,7)(6,8),...","((), (1,2))",1197
640,-3*x^4 + 3*x^3 - 7*x^2 + 3*x - 3,4,1.34559448553445,"[(0.286834423817264 - 1.12397531060423*I, 1), ...",[],x^2 - x + 1/3,"((), (1,2,7,8)(3,6,5,4), (1,3)(2,4)(5,7)(6,8),...","((), (1,2))",1197
5968,3*x^4 + x^3 - 5*x^2 + x + 3,4,1.34559448553445,"[(-1.34559448553445, 1)]",[],x^2 + 1/3*x - 11/3,"((), (1,2)(3,4)(5,7)(6,8), (1,3)(2,4)(5,6)(7,8...","((), (1,2))",-53067
...,...,...,...,...,...,...,...,...,...
3841,x^4 + x^3 + 21*x^2 + x + 1,4,20.9086120551087,"[(-0.477177924427969 - 4.54762721466342*I, 1),...",[],x^2 + x + 19,"((), (1,2,4,3)(5,7,8,6), (1,3,4,2)(5,6,8,7), (...","((), (1,2))",2953125
3803,x^4 + 21*x^2 + 1,4,20.9522724801834,"[(-4.57736523342670*I, 1), (4.57736523342670*I...",[],x^2 + 19,"((), (1,2)(3,4), (1,3)(2,4), (1,4)(2,3))","((), (1,2))",3055504
2668,-x^4 - 21*x^2 - 1,4,20.9522724801834,"[(-4.57736523342670*I, 1), (4.57736523342670*I...",[],x^2 + 19,"((), (1,2)(3,4), (1,3)(2,4), (1,4)(2,3))","((), (1,2))",3055504
2701,-x^4 + 21*x^2 - 1,4,20.9522724801834,"[(-4.57736523342670, 1), (4.57736523342670, 1)]",[],x^2 - 23,"((), (1,2)(3,4), (1,3)(2,4), (1,4)(2,3))","((), (1,2))",3055504
